In [10]:
# install GeoPandas before use
from math import asin, cos, radians, sin, sqrt
import numpy as np
import geopandas as gpd
import pandas as pd
from shapely.geometry import LineString, Point, Polygon

In [ ]:
def generate_OBB(input_path, output_path):
    input_file = gpd.GeoSeries.from_file(input_path)
    gpd.GeoSeries(geom.minimum_rotated_rectangle for geom in input_file).to_file(output_path)

In [ ]:
def HAV(p1, p2):
    """
    calculate distance through coordinates
    p[0] refers to logitude, p[1] refers to latitude in degrees
    """
    lam1, lam2 = radians(p1[0]), radians(p2[0])
    phi1, phi2 = radians(p1[1]), radians(p2[1])
    arg1 = sin((phi1 - phi2) / 2)
    arg2 = sin((lam1 - lam2) / 2)
    arg3 = cos(phi1) * cos(phi2)

    return 2 * 6371000 * asin(sqrt(arg1 * arg1 + arg3 * arg2 * arg2))

In [ ]:
def cut_obb(obb, num1, num2):
    """
    Cut OBB rectangle into num1*num2 grids
    obb -> geom.minimum_rotated_rectangle.exterior.coords
    num1 applies to p1-p2 edge, num2 applies to p2-p3 edge
              num1
    p4 +----------------+ p3
       |                |
       |                | num2
       |                |
    p1 +----------------+ P2
    """
    p1, p2, p3, p4 = Point(obb[0]), Point(obb[1]), Point(obb[2]), Point(obb[3])
    line1, line2 = LineString([p1, p2]), LineString([p4, p3])

    points = []
    # Generate grid points
    for i in range(num1 + 1):
        pointA = line1.interpolate(i / num1, normalized=True)
        pointB = line2.interpolate(i / num1, normalized=True)
        curr_line = LineString([pointA, pointB])
        curr_row = [curr_line.interpolate(i / num2, normalized=True) for i in range(num2 + 1)]
        points.append(curr_row)

    polygons = []
    # Connect points to create grids
    for i in range(num1):
        row1, row2 = points[i], points[i + 1]
        polygons.extend(Polygon([row1[j], row2[j], row2[j + 1], row1[j + 1]]) for j in range(num2))

    return polygons


In [ ]:
input_path = './data/15_label.shp'
interval = 1  # 1 meter
half_interval = interval * 0.5

input_file = gpd.GeoSeries.from_file(input_path)
generate_OBB(input_path, './data/15_label_obb.geojson')

lst = []
for geom in input_file:
    obb = geom.minimum_rotated_rectangle.exterior
    p1, p2, p3, p4, _ = list(obb.coords)
    edge1, edge2 = HAV(p1, p2), HAV(p2, p3)

    cut_num_1 = round(edge1 / interval) + int(edge1 < half_interval)
    cut_num_2 = round(edge2 / interval) + int(edge2 < half_interval)

    lst.append(gpd.GeoSeries(cut_obb(obb.coords, cut_num_1, cut_num_2)))

recs = pd.concat(lst)
recs.to_file(f'./data/cut_rec_{interval}m.geojson')
recs.centroid.to_file(f'./data/cut_rec_centroid_{interval}m.geojson')

In [11]:
# 新版切割函数，这次只在长边方向上切


def cut_polygon(geom, interval):
    """
    Cut OBB rectangle into grids
    """
    obb = geom.minimum_rotated_rectangle.exterior.coords
    p1, p2, p3, p4 = Point(obb[0]), Point(obb[1]), Point(obb[2]), Point(obb[3])
    dist1, dist2 = p1.distance(p2), p2.distance(p3)

    if dist1 > dist2:
        long_edge_1 = LineString([p1, p2])
        long_edge_2 = LineString([p4, p3])
        num = int(round(dist1 / interval) + int(dist1 * 2 < interval))
    else:
        long_edge_1 = LineString([p2, p3])
        long_edge_2 = LineString([p1, p4])
        num = int(round(dist2 / interval) + int(dist2 * 2 < interval))
    # p1 will always on rhe left side

    edge1_points = []
    edge2_points = []
    for i in range(num + 1):
        edge1_points.append(long_edge_1.interpolate(i / num, normalized=True))
        edge2_points.append(long_edge_2.interpolate(i / num, normalized=True))

    polygons = []
    for i in range(num):
        polygons.append(Polygon([edge1_points[i], edge2_points[i], edge2_points[i + 1], edge1_points[i + 1]]))

    return gpd.GeoSeries(polygons)


interval = 0.62  # 1 meter
gdf = gpd.GeoSeries.from_file(r'C:\Users\xianyu\GraduationProject\tobacco_plant_count\data\temp\061301_pred_fat.shp')

result = pd.concat(cut_polygon(geom, interval) for geom in gdf)
result = result.set_crs(epsg=32647)
result.to_file(r'C:\Users\xianyu\GraduationProject\tobacco_plant_count\data\temp\061301_pred_fat_grid.shp')

In [52]:
import geopandas as gpd
from shapely.geometry import Point
from rtree import index

# 假设你有以下的 GeoDataFrame
gdf = gpd.GeoDataFrame(geometry=[Point(1, 1), Point(2, 2), Point(1, 2), Point(3, 3), Point(4, 4), Point(5, 5)])

# 创建一个空的 R-tree 索引
idx = index.Index()

# 填充 R-tree 索引
for i, geom in enumerate(gdf.geometry):
    idx.insert(i, geom.bounds)

# 定义一个阈值，小于该阈值的点对会被选出
threshold = 1.5

# 查询所有距离小于阈值的点对
pairs = set()
for i in range(len(gdf)):
    geom = gdf.geometry[i]
    possible_matches_index = list(idx.nearest((geom.x, geom.y), num_results=4))  # 获得可能的匹配
    possible_matches = gdf.iloc[possible_matches_index]
    precise_matches = possible_matches[possible_matches.distance(geom) < threshold]

    for j, match in precise_matches.iterrows():
        if i != j:
            # 使用 frozenset 可以确保 (a, b) 和 (b, a) 被视为同一对
            pair = frozenset([geom, match.geometry])
            pairs.add(pair)

pairs

{frozenset({<POINT (2 2)>, <POINT (3 3)>}),
 frozenset({<POINT (1 2)>, <POINT (2 2)>}),
 frozenset({<POINT (1 1)>, <POINT (1 2)>}),
 frozenset({<POINT (3 3)>, <POINT (4 4)>}),
 frozenset({<POINT (4 4)>, <POINT (5 5)>}),
 frozenset({<POINT (1 1)>, <POINT (2 2)>})}